In [11]:
import tensorflow as tf
from tensorflow.python.framework import ops
#from keras.layers import Input, Embedding, Dense, Flatten, Dropout, concatenate
from keras.layers import BatchNormalization, SpatialDropout1D, Conv1D
#from keras.callbacks import Callback
#rom keras.models import Model
#from keras.optimizers import Adam


train = "/home/siebert/projects/adtracking-fraud-detection/data/train.csv"


class trainModel(object):
    def __init__(self, batchSize,iterations):
        self.iterations = iterations
        self.batchInput, self.label = self.inputPipeline(batchSize)
        self.app = tf.placeholder(tf.int32, shape=[batchSize])
        self.os = tf.placeholder(tf.int32,shape=[batchSize])
        self.device = tf.placeholder(tf.int32,shape=[batchSize])
        self.learning_rate_placeholder = tf.placeholder(tf.float32, name='learning_rate')
        self.global_step = tf.Variable(0, trainable=False)
        self.label_placeholder = tf.placeholder(tf.int64,shape=(None,1),name="labels")
    
    """
    Read in the features and metadata from the CSV files for each chromosome.
    """
    def readFromCsv(self,fileNameQ):
        #ip,app,device,os,channel,click_time,attributed_time,is_attributed
        reader = tf.TextLineReader(skip_header_lines=1)
        _, csvLine = reader.read(fileNameQ)
        recordDefaults = [[1],[1],[1],[1],[1],[""],[""],[1]]
        recordStr = tf.decode_csv(csvLine, record_defaults=recordDefaults)
        self.label = ops.convert_to_tensor(recordStr[7:8],dtype=tf.int32)
        self.features = [ops.convert_to_tensor(recordStr[0:1],dtype=tf.int32),
                         ops.convert_to_tensor(recordStr[1:2],dtype=tf.int32),
                         ops.convert_to_tensor(recordStr[2:3],dtype=tf.int32),
                         ops.convert_to_tensor(recordStr[3:4],dtype=tf.int32),
                         ops.convert_to_tensor(recordStr[4:5],dtype=tf.int32)]
        return (self.features, self.label)

    """
    Read in the features and metadata from the CSV files for each chromosome.
    """
    def inputPipeline(self, batchSize, num_epochs=None):
        fileNameQ = tf.train.string_input_producer(["/home/siebert/projects/adtracking-fraud-detection/data/train.csv"], shuffle = True)  
        minAfterDequeue = 10000
        capacity = minAfterDequeue + 3 * batchSize
        example, label = self.readFromCsv(fileNameQ)
        exampleBatchStr, labelBatch = tf.train.shuffle_batch([example, label], batch_size=batchSize, capacity=capacity, min_after_dequeue=minAfterDequeue)
        exampleBatch = tf.cast(exampleBatchStr, "int32")
        return (exampleBatchStr, labelBatch)

    ##TRAINING INPUTS
    
    def make_embeddings(self, max_size,inputs):
        embedding_size = 50
        embeddings = tf.Variable(tf.random_uniform([max_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, inputs)
        return embed
        
    def neuralNetwork(self):
        emb_n = 50
        #App embedding
        app_x = self.make_embeddings(99,self.app)
        os_x = self.make_embeddings(99,self.os)
        device_x = self.make_embeddings(997,self.device)
        concat_x = tf.concat([(app_x),(os_x),(device_x)],axis=0)
        s_dout = SpatialDropout1D(0.2)(concat_x)
        
        fl1 = Flatten()(s_dout)
        conv = Conv1D(100, kernel_size=4, strides=1, padding='same')(s_dout)
        fl2 = Flatten()(conv)
        concat = concatenate([(fl1), (fl2)])
        x = Dropout(0.2)(Dense(dense_n,activation='relu')(concat))
        outp = Dense(1,activation='sigmoid')(x)
        return outp
    
    def setup_training(self):
        logits = self.neuralNetwork()
        cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=self.label_placeholder,logits=logits,name='cross_entropy_per_example')
        cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
        tf.add_to_collection('losses',cross_entropy_mean)
        opt = tf.train.AdamOptimizer(self.learning_rate, beta1=0.9, beta2=0.999, epsilon=0.1)
        grads = opt.compute_gradients(cross_entropy_mean, tf.global_variables)
        # Apply gradients.
        apply_gradient_op = opt.apply_gradients(grads, global_step=self.global_step)
        return apply_gradient_op
    
    def train(self):
        with tf.Session() as sess:
            train_op = self.setup_training()
            saver = tf.train.Saver(tf.trainable_variables(), max_to_keep=3)
            summary_op = tf.summary.merge_all()
            gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=args.gpu_memory_fraction)
            sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
            sess.run(tf.global_variables_initializer())
            sess.run(tf.local_variables_initializer())
            summary_writer = tf.summary.FileWriter(log_dir, sess.graph)
            # Start populating the filename queue.
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord,sess=sess)

            #Training iterations
            self.iterationInput = 0
            try:
                for iter in range(self.iterations):
                    #Train with each vector one by one
                    self.iterationInput += 1
                    while not coord.should_stop():
                        #Fill in input data.
                        err_loss,_,step = sess.run([loss,train_op,global_step], feed_dict ={self.app: self.batchInput[1], self.device: self.batchInput[2],self.os: self.BatchInput[3], 
                                                       self.label_placeholder:self.label})
                        
                        print("step :{}, loss :{}".format(step, err_loss))
                        #print(self.label.eval(session = sess))
            except tf.errors.OutOfRangeError:
                print('Done training -- epoch limit reached')

            # When done, ask the threads to stop.
            coord.request_stop()
            coord.join(threads)
        self.coord.request_stop() 

        
model = trainModel(2,100)
model.train()

ValueError: Shape must be at least rank 3 but is rank 2 for 'concat_3' (op: 'ConcatV2') with input shapes: [2,50], [2,50], [2,50], [] and with computed input tensors: input[3] = <2>.